# Test of SNN for Gesture Detection

## Imports

In [ ]:
# SNN
import tonic

# Visualization
import matplotlib.pyplot as plt
from IPython.display import HTML

# Misc
import numpy as np
import numpy.lib.recfunctions as rf

In [ ]:
import os

# Generate a path to save the datasets
data_path = os.path.join(os.getcwd(), "data")
os.makedirs(data_path, exist_ok=True)
train = tonic.datasets.DVSGesture(save_to=data_path, train=True)
test = tonic.datasets.DVSGesture(save_to=data_path, train=False)

In [ ]:
file_name = './data/DVSGesture/ibmGestureTest/user26_led/9.npy'

# each file contains a list of events (x-pos, y-pos, polarity, timestamp).
arr = np.load(file_name)
arr[:, 3] *= 1000  # convert from ms to us
dtype = np.dtype([("x", np.int16), ("y", np.int16), ("p", bool), ("t", np.int64)])
arr = rf.unstructured_to_structured(arr, dtype)

print("A single event:", arr[0], "as (x-pos, y-pos, polarity, timestamp).")

In [ ]:
def to_frames(events):
    # creates dense frames from events by binning them in different ways
    frame_transform = tonic.transforms.ToFrame(
        sensor_size=tonic.datasets.DVSGesture.sensor_size, 
        #time_window=10000)
        n_time_bins=100)
        #event_count=1000)
    return frame_transform(events)

events, label = train[0]
frames = to_frames(events)

print("Train dataset contains", len(train), "samples.")
print("There are", len(events), "events in the selected sample.")
print("A single event:", events[0], "as (x-pos, y-pos, polarity, timestamp).")
print (frames.shape, label)

ani = tonic.utils.plot_animation(frames) # plot one frame
HTML(ani.to_jshtml()) # animate all frames